In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv
/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
random_state=42

In [3]:
%pip install lazypredict

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.preprocessing import LabelEncoder

# pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Data loading

## With no additional data

In [5]:
# train_path = "/kaggle/input/playground-series-s4e2/train.csv"
# test_path = "/kaggle/input/playground-series-s4e2/test.csv"

In [6]:
# df_train = pd.read_csv(train_path)
# df_train = df_train.set_index(["id"])

# df_test = pd.read_csv(test_path)
# df_test = df_test.set_index(["id"])

## With additional data

In [7]:
train_path = "/kaggle/input/playground-series-s4e2/train.csv"
test_path = "/kaggle/input/playground-series-s4e2/test.csv"

In [8]:
df_train = pd.read_csv(train_path)
df_train = df_train.set_index(["id"])

df_test = pd.read_csv(test_path)
df_test = df_test.set_index(["id"])

In [9]:
df_train.shape

(20758, 17)

In [10]:
df_train.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [11]:
df_test.shape

(13840, 16)

In [12]:
df_test.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
id,,,,,,,,,,,,,,,,
20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [13]:
pd.DataFrame([df_train.columns, df_train.dtypes])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
1,object,float64,float64,float64,object,object,float64,float64,object,object,float64,object,float64,float64,object,object,object


In [14]:
df_train.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000
mean,23.841804,1.700245,87.887768,2.445908,2.761332,2.029418,0.981747,0.616756
std,5.688072,0.087312,26.379443,0.533218,0.705375,0.608467,0.838302,0.602113
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,20.000000,1.631856,66.000000,2.000000,3.000000,1.792022,0.008013,0.000000
50%,22.815416,1.700000,84.064875,2.393837,3.000000,2.000000,1.000000,0.573887
75%,26.000000,1.762887,111.600553,3.000000,3.000000,2.549617,1.587406,1.000000
max,61.000000,1.975663,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


### CALC special value

In [15]:
df_train.CALC.value_counts()

CALC
Sometimes     15066
no             5163
Frequently      529
Name: count, dtype: int64

In [16]:
df_test.CALC.value_counts()

CALC
Sometimes     9979
no            3513
Frequently     346
Always           2
Name: count, dtype: int64

In [this comment](https://www.kaggle.com/competitions/playground-series-s4e2/discussion/472597#2631021), the author suggest replace "Always" with "Frequently". 

In [17]:
df_test["CALC"] = np.where(df_test["CALC"] == "Always", "Frequently", df_test["CALC"])

In [18]:
df_test.CALC.value_counts()

CALC
Sometimes     9979
no            3513
Frequently     348
Name: count, dtype: int64

# Pre-processing

## Categorize all "string" columns


In [19]:
gender_encoder = LabelEncoder()
gender_encoder.fit(df_test["Gender"])
gender_category_train = gender_encoder.transform(df_train["Gender"])
gender_category_test = gender_encoder.transform(df_test["Gender"])
df_train["Gender"] = gender_category_train
df_test["Gender"] = gender_category_test

In [20]:
family_history_with_overweight_encoder = LabelEncoder()
family_history_with_overweight_encoder.fit(df_test["family_history_with_overweight"])
family_history_with_overweight_category_train = family_history_with_overweight_encoder.transform(df_train["family_history_with_overweight"])
family_history_with_overweight_category_test = family_history_with_overweight_encoder.transform(df_test["family_history_with_overweight"])
df_train["family_history_with_overweight"] = family_history_with_overweight_category_train
df_test["family_history_with_overweight"] = family_history_with_overweight_category_test

In [21]:
FAVC_encoder = LabelEncoder()
FAVC_encoder.fit(df_test["FAVC"])
FAVC_category_train = FAVC_encoder.transform(df_train["FAVC"])
FAVC_category_test = FAVC_encoder.transform(df_test["FAVC"])
df_train["FAVC"] = FAVC_category_train
df_test["FAVC"] = FAVC_category_test

In [22]:
CAEC_encoder = LabelEncoder()
CAEC_encoder.fit(df_test["CAEC"])
CAEC_category_train = CAEC_encoder.transform(df_train["CAEC"])
CAEC_category_test = CAEC_encoder.transform(df_test["CAEC"])
df_train["CAEC"] = CAEC_category_train
df_test["CAEC"] = CAEC_category_test

In [23]:
SMOKE_encoder = LabelEncoder()
SMOKE_encoder.fit(df_test["SMOKE"])
SMOKE_category_train = SMOKE_encoder.transform(df_train["SMOKE"])
SMOKE_category_test = SMOKE_encoder.transform(df_test["SMOKE"])
df_train["SMOKE"] = SMOKE_category_train
df_test["SMOKE"] = SMOKE_category_test

In [24]:
SCC_encoder = LabelEncoder()
SCC_encoder.fit(df_test["SCC"])
SCC_category_train = SCC_encoder.transform(df_train["SCC"])
SCC_category_test = SCC_encoder.transform(df_test["SCC"])
df_train["SCC"] = SCC_category_train
df_test["SCC"] = SCC_category_test

In [25]:
CALC_encoder = LabelEncoder()
CALC_encoder.fit(df_train["CALC"])
CALC_category_train = CALC_encoder.transform(df_train["CALC"])
CALC_category_test = CALC_encoder.transform(df_test["CALC"])
df_train["CALC"] = CALC_category_train
df_test["CALC"] = CALC_category_test

In [26]:
MTRANS_encoder = LabelEncoder()
MTRANS_encoder.fit(df_test["MTRANS"])
MTRANS_category_train = MTRANS_encoder.transform(df_train["MTRANS"])
MTRANS_category_test = MTRANS_encoder.transform(df_test["MTRANS"])
df_train["MTRANS"] = MTRANS_category_train
df_test["MTRANS"] = MTRANS_category_test

In [27]:
df_train.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,2.763573,0,0.000000,0.976473,1,3,Overweight_Level_II
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,2,0,Normal_Weight
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,1.910378,0,0.866045,1.673584,2,3,Insufficient_Weight
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,1.674061,0,1.467863,0.780199,1,3,Obesity_Type_III
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,1.979848,0,1.967973,0.931721,1,3,Overweight_Level_II


Label encoding

In [28]:
NObeyesdad_encoder = LabelEncoder()
NObeyesdad_encoder.fit(df_train["NObeyesdad"])
NObeyesdad_category_train = NObeyesdad_encoder.transform(df_train["NObeyesdad"])
df_train["NObeyesdad"] = NObeyesdad_category_train

In [29]:
df_train.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,2.763573,0,0.000000,0.976473,1,3,6
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,2,0,1
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,1.910378,0,0.866045,1.673584,2,3,0
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,1.674061,0,1.467863,0.780199,1,3,4
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,1.979848,0,1.967973,0.931721,1,3,6


In [30]:
df_test.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
id,,,,,,,,,,,,,,,,
20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,2,0,2.825629,0,0.855400,0.000000,1,3
20759,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,2,0,3.000000,0,1.000000,0.000000,1,3
20760,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,2,0,2.621877,0,0.000000,0.250502,1,3
20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,2,0,2.786417,0,0.094851,0.000000,1,3
20762,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,2,0,2.653531,0,0.000000,0.741069,1,3


## Feature engineering

In [31]:
# BMI = Weight / height*height
df_train["BMI"] = df_train["Weight"] / (df_train["Height"] * df_train["Height"])
df_test["BMI"] = df_test["Weight"] / (df_test["Height"] * df_test["Height"])

# WIR
df_train["WIR"] = df_train["Weight"] / df_train["CH2O"]
df_test["WIR"] = df_test["Weight"] / df_test["CH2O"]

# PAL
df_train["PAL"] = df_train["TUE"] - df_train["FAF"]
df_test["PAL"] = df_test["TUE"] - df_test["FAF"]

# MH
df_train["MH"] = df_train["FCVC"] + df_train["NCP"]
df_test["MH"] = df_test["FCVC"] + df_test["NCP"]

# HNH
df_train["HNH"] = df_train["FCVC"] + df_train["FAVC"]
df_test["HNH"] = df_test["FCVC"] + df_test["FAVC"]

# TUS
df_train["TUS"] = df_train["Age"] * df_train["TUE"]
df_test["TUS"] = df_test["Age"] * df_test["TUE"]

# drop height and weight
# df_train.drop(["Height", "Weight"], axis=1, inplace=True)
# df_test.drop(["Height", "Weight"], axis=1, inplace=True)

In [32]:
df_train.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,TUE,CALC,MTRANS,NObeyesdad,BMI,WIR,PAL,MH,HNH,TUS
id,,,,,,,,,,,,,,,,,,,,,
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,...,0.976473,1,3,6,28.259565,29.552304,0.976473,4.983297,3.000000,23.867940
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,...,1.000000,2,0,1,23.422091,28.500000,0.000000,5.000000,3.000000,18.000000
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,...,1.673584,2,3,0,17.126706,26.259596,0.807539,3.292219,2.880534,30.124512
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,...,0.780199,1,3,4,44.855798,78.417006,-0.687664,6.000000,4.000000,16.347304
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,...,0.931721,1,3,6,25.599151,47.376392,-1.036252,4.651136,3.679664,29.480660


In [33]:
df_test.head(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,FAF,TUE,CALC,MTRANS,BMI,WIR,PAL,MH,HNH,TUS
id,,,,,,,,,,,,,,,,,,,,,
20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,2,0,...,0.855400,0.000000,1,3,35.315411,42.696397,-0.855400,5.938616,3.938616,0.000000
20759,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,2,0,...,1.000000,0.000000,1,3,25.781250,22.000000,-1.000000,3.000000,3.000000,0.000000
20760,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,2,0,...,0.000000,0.250502,1,3,41.324115,42.565137,0.250502,6.000000,4.000000,6.513052
20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,2,0,...,0.094851,0.000000,1,3,42.976937,37.205169,-0.094851,4.977909,3.000000,0.000000
20762,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,2,0,...,0.000000,0.741069,1,3,39.584143,39.507866,0.741069,6.000000,4.000000,19.267794


## Data Visualization

In [34]:
# sns.pairplot(df_train[["BMI", "Age", "NObeyesdad"]], hue="NObeyesdad", height=10)
# plt.show()

In [35]:
# sns.plot(df_train[["Gender", "NObeyesdad"]], hue="NObeyesdad")
# plt.show()

# Model overview

In [36]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report

In [37]:
kf = StratifiedKFold(n_splits = 5, random_state=random_state, shuffle=True)

In [38]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

In [39]:
def CV(clf, X, y, kf):
    metric = []
    
    for i, (t_ind, v_ind) in enumerate(kf.split(X, y)):
        train_X, validate_X = X.iloc[t_ind], X.iloc[v_ind]
        train_y, validate_y = y.iloc[t_ind], y.iloc[v_ind]

        model = clf.fit(train_X, train_y)
        
        preds = model.predict(validate_X)
        
        score = accuracy_score(validate_y, preds)
        
        print(f"Finished fold {i + 1}")
        
        print(classification_report(validate_y, preds))
        
        metric.append(score)
        
    return np.mean(metric), np.std(metric)

Using lazypredict to get best performance models

In [40]:
# X, y = df_train.drop(["NObeyesdad"], axis=1), df_train["NObeyesdad"]
# for i, (t_ind, v_ind) in enumerate(kf.split(X, y)):
#     train = df_train.iloc[t_ind]
#     train_X = train.drop(["NObeyesdad"], axis=1)
#     train_y = train["NObeyesdad"]
    
#     validate = df_train.iloc[v_ind]
#     validate_X = validate.drop(["NObeyesdad"], axis=1)
#     validate_y = validate["NObeyesdad"]
    
#     models, predictions = clf.fit(train_X, validate_X, train_y, validate_y)
    
#     print(f"Fold {i + 1}")
#     print(models)

I'll get top 8 predictor (respectively):
1. LGBMClassifier
2. XGBClassifier
3. RandomForestClassifier
4. BaggingClassifier
5. ExtraTreesClassifier
6. SVC
7. LogisticRegression
8. DecisionTreeClassifier

## Grid Search to find best parameters

In [41]:
train_X = df_train.drop(["NObeyesdad"], axis=1)
train_y = df_train["NObeyesdad"]

In [42]:
import optuna

In [43]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [44]:
def xgb_objective(trial):
    kf = StratifiedKFold(n_splits=5, 
                         shuffle=True,
                         random_state=random_state)
    scores = []
    
    # List of suggested parameters
    param = {
        "verbosity": 0,
        "objective": "multi:softprob",
        # use exact for small dataset.
        "tree_method": trial.suggest_categorical("tree_method", ["exact", "approx", "hist"]),
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }
    
    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
        
    for train_idx, val_idx in kf.split(train_X, train_y):
        train_X_fold, train_y_fold = train_X.iloc[train_idx], train_y.iloc[train_idx]
        val_X_fold, val_y_fold = train_X.iloc[val_idx], train_y.iloc[val_idx]
        
        model = XGBClassifier(
            random_state=42,
            **param
        )
        
        model.fit(train_X_fold, train_y_fold)
        
        preds = model.predict(val_X_fold)
        
        accuracy_fold = accuracy_score(val_y_fold, preds)
        
        scores.append(accuracy_fold)
    
    return np.mean(scores)

In [45]:
# study_xgb = optuna.create_study(direction="maximize")
# study_xgb.optimize(xgb_objective, n_trials=20)

# print('Number of finished trials:', len(study_xgb.trials))
# print('Best trial:', study_xgb.best_trial.params)

In [46]:
def lgbm_objective(trial):
    kf = StratifiedKFold(n_splits=5, 
                         shuffle=True,
                         random_state=random_state)
    scores = []
    
    # List of suggested parameters
    param = {
        "objective": "multiclass",
        "metric": "softmax",
        "verbosity": -1,
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "rf", "dart"]),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
        
    for train_idx, val_idx in kf.split(train_X, train_y):
        train_X_fold, train_y_fold = train_X.iloc[train_idx], train_y.iloc[train_idx]
        val_X_fold, val_y_fold = train_X.iloc[val_idx], train_y.iloc[val_idx]
        
        model = LGBMClassifier(
            random_state=42,
            **param
        )
        
        model.fit(train_X_fold, train_y_fold)
        
        preds = model.predict(val_X_fold)
        
        accuracy_fold = accuracy_score(val_y_fold, preds)
        
        scores.append(accuracy_fold)
    
    return np.mean(scores)

In [47]:
# study_lgbm = optuna.create_study(direction="maximize")
# study_lgbm.optimize(lgbm_objective, n_trials=20)

# print('Number of finished trials:', len(study_lgbm.trials))
# print('Best trial:', study_lgbm.best_trial.params)

# Try predicting

In [48]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

In [49]:
# final_classifier = VotingClassifier(
#     estimators = [
#         ("extra_trees", ExtraTreesClassifier(random_state=random_state)),
#         ("rf", RandomForestClassifier(random_state=random_state)),
# #         ("dt", DecisionTreeClassifier(random_state=random_state)),
# #         ("bagging", BaggingClassifier(random_state=random_state)),
# #         ("svc", SVC(random_state=random_state)),
# #         ("lr", LogisticRegression(random_state=random_state)),
#         ("xgb", XGBClassifier(random_state=random_state)),
#         ("lgbm", LGBMClassifier(random_state=random_state))
#     ]
# )

final_classifier = StackingClassifier(
    estimators = [
#         ("extra_trees", ExtraTreesClassifier(random_state=random_state)),
#         ("rf", RandomForestClassifier(random_state=random_state)),
#         ("dt", DecisionTreeClassifier(random_state=random_state)),
#         ("bagging", BaggingClassifier(random_state=random_state)),
#         ("svc", SVC(random_state=random_state)),
#         ("lr", LogisticRegression(random_state=random_state)),
        ("xgb", XGBClassifier(random_state=random_state, **{
            "verbosity": 0,
            "objective": "multi:softprob",
            'tree_method': 'hist', 
            'booster': 'dart', 
            'lambda': 0.0010046902217712437, 
            'alpha': 2.3228898994574606e-05, 
            'subsample': 0.8430271830704958, 
            'colsample_bytree': 0.6062771078777813, 
            'max_depth': 7, 
            'min_child_weight': 6, 
            'eta': 0.13030898540211466, 
            'gamma': 1.9167254782290146e-05, 
            'grow_policy': 'lossguide', 
            'sample_type': 'uniform', 
            'normalize_type': 'forest', 
            'rate_drop': 8.616905977778094e-06, 
            'skip_drop': 0.008301542112391098
        })),
        ("lgbm", LGBMClassifier(random_state=random_state, **{
            "objective": "multiclass",
            "metric": "softmax",
            "verbosity": -1,
            'boosting_type': 'gbdt', 
            'lambda_l1': 7.87867319097297e-07, 
            'lambda_l2': 0.13088250802149265, 
            'num_leaves': 35, 
            'feature_fraction': 0.5013740302606244, 
            'bagging_fraction': 0.7779022790702624, 
            'bagging_freq': 2, 
            'min_child_samples': 89}))
    ]
)

## CV

In [50]:
acc_mean, acc_std = CV(final_classifier, train_X, train_y, kf)

print(f"Mean accuracy = {acc_mean}, std accuracy = {acc_std}")

Finished fold 1
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       505
           1       0.90      0.89      0.90       616
           2       0.91      0.90      0.90       582
           3       0.98      0.98      0.98       650
           4       1.00      1.00      1.00       809
           5       0.78      0.77      0.78       485
           6       0.82      0.83      0.82       505

    accuracy                           0.91      4152
   macro avg       0.90      0.90      0.90      4152
weighted avg       0.91      0.91      0.91      4152

Finished fold 2
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       505
           1       0.87      0.89      0.88       616
           2       0.89      0.88      0.89       582
           3       0.96      0.96      0.96       649
           4       1.00      0.99      1.00       809
           5       0.81      0.74      0.77   

## Fit the final classifier

In [51]:
final_classifier.fit(train_X, train_y)

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(alpha=2.3228898994574606e-05,
                                              base_score=None, booster='dart',
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.6062771078777813,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eta=0.13030898540211466,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=1.9167254782...
                                              min_child_weight=6, missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, ...)),
                               ('lgbm',
                                LGBMClassifier(bagging_fraction=0.7779022790702624,
                                               bagging_freq=2,
                                               feature_fraction=0.5013740302606244,
                                               lambda_l1=7.87867319097297e-07,
                                               lambda_l2=0.13088250802149265,
                                               metric='softmax',
                                               min_child_samples=89,
                                               num_leaves=35,
                                               objective='multiclass',
                                               random_state=42,
                                               verbosity=-1))])

## Predict the validation set

In [52]:
# eval_X, eval_y = df_eval.drop(["NObeyesdad"], axis=1), df_eval["NObeyesdad"]
# eval_X, eval_y = df_eval[["Gender", "Age", "BMI"]], df_eval["NObeyesdad"]

# val_predictions = final_classifier.predict(eval_X)

In [53]:
# print(classification_report(eval_y, val_predictions))

## Predict the final test set

In [54]:
predictions = final_classifier.predict(df_test)

In [55]:
predictions

array([3, 5, 4, ..., 0, 1, 3])

In [56]:
NObeyesdad_encoder.inverse_transform(predictions)

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype=object)

In [57]:
df_test.index.values

array([20758, 20759, 20760, ..., 34595, 34596, 34597])

In [58]:
submission_df = pd.DataFrame({
    "id": df_test.index.values,
    "NObeyesdad": NObeyesdad_encoder.inverse_transform(predictions)
})

In [59]:
submission_df

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [60]:
submission_df.to_csv("submission.csv", index=False)